In [ ]:
#!/usr/bin/env python

# UV10_animation.ipynb

In [1]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import netCDF4
import glob
import os
from pdb import set_trace as stop
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage import median_filter
from scipy.ndimage import label
from matplotlib import cm
from scipy import ndimage
import random
import scipy
import pickle
import datetime
import pandas as pd
import subprocess
from calendar import monthrange
import pandas as pd
import datetime
import sys 
import shapefile as shp
import matplotlib.path as mplPath
from scipy.stats import norm
import matplotlib.gridspec as gridspec
# from mpl_toolkits.basemap import Basemap, cm
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.gridspec as gridspec
from pylab import *
import string
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import shapefile
from calendar import monthrange
from tqdm import tqdm
import seaborn as sns
import xarray as xr


def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

def runningMeanFast(x, N):
    return np.convolve(x, np.ones((N,))/N)[(N-1):]

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    km = 6367 * c
    return km


###  Read CONUS404 data

In [20]:
YYYY = 1980 #int(sys.argv[1])


TimeHH = pd.date_range(datetime.datetime(YYYY, 1, 1,0), end=datetime.datetime(YYYY, 12, 31,23), freq='h')
TimeMM = pd.date_range(datetime.datetime(YYYY, 1, 1,0), end=datetime.datetime(YYYY, 12, 31,23), freq='m')
PlotDir = '/glade/campaign/mmm/c3we/prein/CONUS404/plots/UV10_animation/'+ str(TimeHH[0])[:10] + '-' +str(TimeHH[-1])[:10] + '/'
if not os.path.exists(PlotDir):
    os.makedirs(PlotDir)
    
# WRF
WRFdir = '/glade/campaign/ncar/USGS_Water/CONUS404/' 
WRFvar = ['U10','V10']

lon_start = 0
lon_stop = 1366

out_dir = '/glade/campaign/mmm/c3we/prein/CONUS404/data/MonthlyData/uv10/'

In [3]:
ncid=Dataset('/glade/campaign/mmm/c3we/prein/CONUS404/data/MonthlyData/wrfout_conus404_constants.nc', mode='r')
lon_wrf=np.squeeze(ncid.variables['XLONG'][:,:,lon_start:lon_stop])
lat_wrf=np.squeeze(ncid.variables['XLAT'][:,:,lon_start:lon_stop])
ncid.close()
lon_wrf[lon_wrf > 180] = lon_wrf[lon_wrf > 180]-360

In [4]:
wrf_data = np.zeros((len(TimeHH),lon_wrf.shape[0],lon_wrf.shape[1],len(WRFvar)), dtype=np.float16); wrf_data[:] = np.nan

DataFolder = '/glade/campaign/mmm/c3we/prein/CONUS404/data/MonthlyData/'
for mm in tqdm(range(len(TimeMM))[:1]):
    YYYY = TimeMM[mm].year
    MM = TimeMM[mm].month
    rgiHours = (TimeHH.year == YYYY) & (TimeHH.month == MM)
    File_act = DataFolder + 'U10_'+str(YYYY)+str(MM).zfill(2)+'_CONUS404.nc'
    ds_disk = xr.open_dataset(File_act)
    wrf_data[rgiHours,:,:,0] = np.array(ds_disk['U10'][:,:,lon_start:lon_stop], dtype=np.float16)
    File_act = DataFolder + 'V10_'+str(YYYY)+str(MM).zfill(2)+'_CONUS404.nc'
    ds_disk = xr.open_dataset(File_act)
    wrf_data[rgiHours,:,:,1] = np.array(ds_disk['V10'][:,:,lon_start:lon_stop], dtype=np.float16)
    

100%|██████████| 1/1 [02:45<00:00, 165.29s/it]


In [8]:
wrf_uv10 = (wrf_data[:,:,:,0]**2 + wrf_data[:,:,:,1]**2)**0.5

In [10]:
# calculate monthly statistics
metrics = ['<1','mean','max']
wrf_uv10_mean = np.zeros((12,lon_wrf.shape[0],lon_wrf.shape[1], len(metrics)), dtype=np.float16); wrf_uv10_mean[:] = np.nan

for mm in tqdm(range(12)[:1]):
    time_sel = np.isin(TimeHH.month, [mm+1])
    data_mm = wrf_uv10[time_sel,:,:]
    for me in range(len(metrics)):
        if metrics[me] == '<1':
            wrf_uv10_mean[mm,:,:,me] = np.nansum(data_mm[:,:,:] < 1, axis=0)/np.nansum(time_sel)
        elif metrics[me] == 'mean':
            wrf_uv10_mean[mm,:,:,me] = np.nanmean(data_mm[:,:,:], axis=0)
        elif metrics[me] == 'max':
            wrf_uv10_mean[mm,:,:,me] = np.nanmax(data_mm[:,:,:], axis=0)
        else:
            continue


100%|██████████| 1/1 [00:35<00:00, 35.82s/it]


In [21]:
np.savez(out_dir + 'UV10_'+str(YYYY)+'_CONUS404.npz',
        wrf_uv10 = wrf_uv10,
        wrf_uv10_mean = wrf_uv10_mean,
        metrics = metrics,
        TimeHH = TimeHH,
        lon_wrf = lon_wrf,
        lat_wrf = lat_wrf)

KeyboardInterrupt: 